In [4]:
# 硬投票分类器
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard')

from sklearn.metrics import accuracy_score

for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__,accuracy_score(y_pred,y_val))

LogisticRegression 0.864
RandomForestClassifier 0.904
SVC 0.896
VotingClassifier 0.912


In [5]:
# bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,bootstrap=True,n_jobs=-1)
tree_clf = DecisionTreeClassifier()

for clf in (tree_clf, bag_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__,accuracy_score(y_pred,y_val))

DecisionTreeClassifier 0.816
BaggingClassifier 0.904


In [6]:
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1) , n_estimators=300,
    learning_rate=0.2)
for clf in (tree_clf, ada_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__,accuracy_score(y_pred,y_val))

DecisionTreeClassifier 0.84
AdaBoostClassifier 0.912


In [7]:
# xgboost

import xgboost

xgb_reg = xgboost.XGBClassifier()
for clf in (tree_clf, xgb_reg):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__,accuracy_score(y_pred,y_val))

DecisionTreeClassifier 0.848
[20:34:01] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier 0.872
